In [ ]:
import numpy as np
import os
os.chdir("..")

from helpers import *
from helpers_perso import *
from nan_imputation import *
from one_hot_encoding import *
from implementations import *
from standardization import *

%load_ext autoreload
%autoreload 2

Load data

In [ ]:
data_path = os.path.join(os.getcwd(), "data", "dataset")
x_train, x_test, y_train, train_ids, test_ids = load_csv_data(data_path)
print("Data loaded successfully!")


## Variance across columns

In [ ]:
import matplotlib.pyplot as plt

column_variances = np.nanvar(x_train, axis=0)

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# First subplot: Full range of variances with more bins
axes[0].hist(column_variances, bins=100, edgecolor='black')
axes[0].set_title("Distribution of Variance Across Columns (Full Range)")
axes[0].set_xlabel("Variance")
axes[0].set_ylabel("Frequency")

# Second subplot: Focus on variance between 0 and 0.5
axes[1].hist(column_variances, bins=20, range=(0, 1000), edgecolor='black')
axes[1].set_title("Distribution of Variance Across Columns (0 to 1000)")
axes[1].set_xlabel("Variance")
axes[1].set_ylabel("Frequency")

# Third subplot: Focus on variance between 1 and 200
axes[2].hist(column_variances, bins=100, range=(1, 200), edgecolor='black')
axes[2].set_title("Distribution of Variance Across Columns (1 to 200)")
axes[2].set_xlabel("Variance")
axes[2].set_ylabel("Frequency")

# Display the plots
plt.tight_layout()
plt.show()

## Handling columns containing Nan

Proportion of Nan values in Nan-containing values

In [ ]:
import matplotlib.pyplot as plt

# Calculate the proportion of NaN values in each column
nan_proportions = np.isnan(x_train).mean(axis=0)

# Print the total number of columns plotted
total_columns = nan_proportions.size
print(f"Total number of columns plotted: {total_columns}")

# Print the number of columns containing NaN values
num_columns_with_nans = np.sum(nan_proportions > 0)
print(f"Number of columns containing NaN values: {num_columns_with_nans}")

# Define the bins for the histogram
bins = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]

# Calculate the histogram
hist, bin_edges = np.histogram(nan_proportions, bins=bins)

# Create the bar plot
plt.bar(range(len(hist)), hist, tick_label=[f'{int(b*100)}-{int(bins[i+1]*100)}%' for i, b in enumerate(bins[:-1])])
plt.xlabel('Proportion of NaN values')
plt.ylabel('Number of columns')
plt.title('Number of columns containing a proportion of NaN values')

# Rotate the x-axis tick labels to vertical
plt.xticks(rotation=90)

plt.show()

Thus not reasonnable to exclude columns containing Nan values
Choice : remove columns with Nan proportion superior to 80 % ?

In [42]:
# Clean all arrays by removing columns containing NaN values
x_train_cleaned, deleted_indices = remove_nan_features(x_train, 0.8)

adapted_x_test = np.delete(x_test, deleted_indices, axis=1)


Percentage of columns to delete (NaN proportion superior to 80.0 %): 36.14%
Data cleaned successfully!
Original shape of x_train: (328135, 321)
Cleaned shape of x_train: (328135, 205)


In [43]:
integer_columns, non_integer_columns = identify_integer_columns(x_train_cleaned)

assert len(integer_columns) + len(non_integer_columns) == x_train_cleaned.shape[1]


# Print the integer columns
print(f"Number of columns containing only integer values: {len(integer_columns)}")

# Count the number of columns in integer_columns that contain at least one zero
num_columns_with_zero = sum(np.any(x_train_cleaned[:, col] == 0) for col in integer_columns)

# Print the result
print(f"Percentage of integer columns that contain at least one zero: {num_columns_with_zero/len(integer_columns)*100:.2f}%")    

Number of columns containing only integer values: 182
Percentage of integer columns that contain at least one zero: 13.19%


(If only contains integers and no zeroes (=encoded), encode Nan as 0)

In [44]:
x_train_cleaned_without_nans = encode_nan_integer_columns(x_train_cleaned, replacement_value='mode')
x_train_cleaned_without_nans = encode_nan_continuous_columns(x_train_cleaned_without_nans, replacement_value='mode')

assert np.isnan(x_train_cleaned_without_nans).sum() == 0
assert x_train_cleaned.shape == x_train_cleaned_without_nans.shape

adapted_x_test_without_nans = encode_nan_integer_columns(adapted_x_test, replacement_value='mode')
adapted_x_test_without_nans = encode_nan_continuous_columns(adapted_x_test_without_nans, replacement_value='mode')

assert np.isnan(adapted_x_test_without_nans).sum() == 0
assert adapted_x_test.shape == adapted_x_test_without_nans.shape

Column 9 has been encoded with NaNs as the mode 1.0
Column 10 has been encoded with NaNs as the mode 1.0
Column 11 has been encoded with NaNs as the mode 1.0
Column 12 has been encoded with NaNs as the mode 2.0
Column 13 has been encoded with NaNs as the mode 2.0
Column 14 has been encoded with NaNs as the mode 1.0
Column 15 has been encoded with NaNs as the mode 1.0
Column 16 has been encoded with NaNs as the mode 1.0
Column 17 has been encoded with NaNs as the mode 1.0
Column 18 has been encoded with NaNs as the mode 2.0
Column 19 has been encoded with NaNs as the mode 1.0
Column 20 has been encoded with NaNs as the mode 1.0
Column 21 has been encoded with NaNs as the mode 2.0
Column 22 has been encoded with NaNs as the mode 2.0
Column 23 has been encoded with NaNs as the mode 2.0
Column 24 has been encoded with NaNs as the mode 88.0
Column 26 has been encoded with NaNs as the mode 88.0
Column 29 has been encoded with NaNs as the mode 2.0
Column 30 has been encoded with NaNs as the m

In [ ]:
# Calculate the number of columns that do not contain only integer values
num_non_integer_columns = len(non_integer_columns)

# Print the result
print(f"Number of columns that do not contain only integer values: {num_non_integer_columns}")

## One-hot encoding

In columns containing only integer values, number of unique values

In [ ]:
import matplotlib.pyplot as plt

unique_value_counts = np.array([len(np.unique(x_train_cleaned[:, col])) for col in integer_columns])

# Create 20 bins based on the range of unique value counts
max_unique = unique_value_counts.max() if unique_value_counts.size > 0 else 0
bins = np.linspace(0, max_unique, 21)  # 21 edges for 20 bins
bin_labels = [f'{int(b)}-{int(bins[i+1])}' for i, b in enumerate(bins[:-1])]

# Count how many columns fall into each bin
binned_counts = np.histogram(unique_value_counts, bins=bins)[0]

# Create the bar plot
plt.figure(figsize=(10, 6))
plt.bar(bin_labels, binned_counts, width=0.6, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Unique Values in Columns')
plt.ylabel('Number of Columns')
plt.title('Columns Grouped by Number of Unique Values (20 Bins)')
plt.xticks(rotation=45)
plt.tight_layout()
# Create the second set of bins for values between 2000 and 0
bins_2000 = np.linspace(0, 2000, 21)  # 21 edges for 20 bins
bin_labels_2000 = [f'{int(b)}-{int(bins_2000[i+1])}' for i, b in enumerate(bins_2000[:-1])]

# Count how many columns fall into each bin for the second set of bins
binned_counts_2000 = np.histogram(unique_value_counts, bins=bins_2000)[0]

# Create the second bar plot
plt.figure(figsize=(10, 6))
plt.bar(bin_labels_2000, binned_counts_2000, width=0.6, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Unique Values in Columns (0-2000)')
plt.ylabel('Number of Columns')
plt.title('Columns Grouped by Number of Unique Values (20 Bins, 0-2000)')
plt.xticks(rotation=45)
plt.tight_layout()

# Create the third set of bins for values between 0 and 100
bins_100 = np.linspace(0, 100, 21)  # 21 edges for 20 bins
bin_labels_100 = [f'{int(b)}-{int(bins_100[i+1])}' for i, b in enumerate(bins_100[:-1])]

# Count how many columns fall into each bin for the third set of bins
binned_counts_100 = np.histogram(unique_value_counts, bins=bins_100)[0]

# Create the third bar plot
plt.figure(figsize=(10, 6))
plt.bar(bin_labels_100, binned_counts_100, width=0.6, edgecolor='black', alpha=0.7)
plt.xlabel('Number of Unique Values in Columns (0-100)')
plt.ylabel('Number of Columns')
plt.title('Columns Grouped by Number of Unique Values (20 Bins, 0-100)')
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()


Given number of unique value mainly in 0-5 range, lets say it's categorical if in this range

In [45]:
categorical_threshold = 5

# Step 1: Compute unique value counts for each integer column
unique_value_counts = np.array([len(np.unique(x_train_cleaned[:, col])) for col in integer_columns])

# Step 2: Identify categorical and non-categorical features based on the threshold
indexes_categorical_features = [integer_columns[i] for i, count in enumerate(unique_value_counts) if count <= categorical_threshold]
indexes_non_categorical_features = [integer_columns[i] for i in range(len(unique_value_counts)) if integer_columns[i] not in indexes_categorical_features]

assert len(indexes_categorical_features) + len(indexes_non_categorical_features) == len(unique_value_counts)
assert unique_value_counts.size == len(integer_columns)

indexes_non_categorical_features.extend(non_integer_columns)


## Standardization

In [46]:
# x_standardized = standardize_columns(x_train_cleaned_without_nans, range(x_train_cleaned_without_nans.shape[1]))

x_standardized = standardize_columns(x_train_cleaned_without_nans, indexes_non_categorical_features)

x_test_standardized = standardize_columns(adapted_x_test_without_nans, indexes_non_categorical_features)

Binary encoding

In [47]:
# encoded_x_train = binary_encode_columns(x_standardized, indexes_categorical_features)

encoded_x_train, encoded_x_test = consistent_binary_encode(x_standardized, x_test_standardized, indexes_categorical_features)

In [48]:
print(encoded_x_train.shape)
print(encoded_x_test.shape)

(328135, 291)
(109379, 291)


## Finding best parameters


In [49]:
# X_ici = x_train_cleaned_without_nans
# X_ici = standardize_columns(X_ici, indexes_non_categorical_features)

X_ici = encoded_x_train

# # X_ici = x_train_cleaned





y_ici = y_train

# linear regression
initial_w = np.zeros(X_ici.shape[1])
max_iters = 30
gamma = 0.01
# print (least_squares(y_ici, X_ici))
w, loss = mean_squared_error_gd(y_ici, X_ici, initial_w, max_iters, gamma)

# percentages_to_drop = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# nan_values_for_integer_columns = ['mode', 'upper', 'zero']
# nan_values_for_continuous_columns = ['mean', 'mode', 'zero']



GD iter. 0/29: loss=0.28796094552037377, w0=3.1937394486283e-05, w1=3.7822689140182695e-05
GD iter. 1/29: loss=0.20462583590895084, w0=6.483885073931895e-05, w1=6.815606421484738e-05
GD iter. 2/29: loss=0.1715095077404318, w0=9.759493856512131e-05, w1=9.357042566203736e-05
GD iter. 3/29: loss=0.15803033802969904, w0=0.0001295640075152702, w1=0.00011566807066494498
GD iter. 4/29: loss=0.15226677786053286, w0=0.00016039281271903623, w1=0.0001354483363872032
GD iter. 5/29: loss=0.14956674532952566, w0=0.00018990481312281468, w1=0.0001535352914066538
GD iter. 6/29: loss=0.1481107782823617, w0=0.00021803068712045306, w1=0.00017031971736389262
GD iter. 7/29: loss=0.14718371744570138, w0=0.0002447651802468456, w1=0.00018604760974367428
GD iter. 8/29: loss=0.1465013895658475, w0=0.0002701403719804014, w1=0.00020087531103884037
GD iter. 9/29: loss=0.1459482336374731, w0=0.0002942091759896229, w1=0.00021490382820801737
GD iter. 10/29: loss=0.1454750060478609, w0=0.0003170352142943987, w1=0.00022

In [50]:
from predict_labels import *
# adapted_x_test = np.delete(x_test, deleted_indices, axis=1)
# standardized_x_test = standardize_columns(adapted_x_test, indexes_non_categorical_features)
# encoded_x_test = binary_encode_columns(adapted_x_test, indexes_categorical_features)
# print(standardized_x_test.shape)
# print(x_standardized.shape)

# print(encoded_x_test.shape)
# print(encoded_x_train.shape)
y_test = predict_classification(encoded_x_test,w)
print(y_test)

[-1 -1 -1 ... -1 -1 -1]


percentage of Nan to drop

In [ ]:
best_loss = np.inf
best_percentage_to_drop = None
best_nan_value_for_integer_columns = None
best_nan_value_for_continuous_columns = None

for percentage_to_drop in percentages_to_drop:
    for nan_value_for_continuous_columns in nan_values_for_continuous_columns:
        for nan_value_for_integer_columns in nan_values_for_integer_columns:
    
            x_train_cleaned = remove_nan_features(x_train, percentage_to_drop)

            x_train_cleaned_without_nans = encode_nan_integer_columns(x_train_cleaned, replacement_value=nan_value_for_integer_columns)
            x_train_cleaned_without_nans = encode_nan_continuous_columns(x_train_cleaned_without_nans, replacement_value=nan_value_for_continuous_columns)

            assert np.isnan(x_train_cleaned_without_nans).sum() == 0
            assert x_train_cleaned.shape == x_train_cleaned_without_nans.shape

            x_standardized = standardize_columns(x_train_cleaned_without_nans, range(x_train_cleaned_without_nans.shape[1]))

            initial_w = np.zeros(X_ici.shape[1])
            max_iters = 100
            gamma = 0.01

            w, loss = mean_squared_error_gd(y_ici, X_ici, initial_w, max_iters, gamma)

            if loss < best_loss:
                best_loss = loss
                best_percentage_to_drop = percentage_to_drop
                best_nan_value_for_integer_columns = nan_value_for_integer_columns
                best_nan_value_for_continuous_columns = nan_value_for_continuous_columns

print(f"Best loss: {best_loss}")    
print(f"Best percentage to drop: {best_percentage_to_drop}")
print(f"Best nan value for integer columns: {best_nan_value_for_integer_columns}")
print(f"Best nan value for continuous columns: {best_nan_value_for_continuous_columns}")
    
    